In [7]:
library(phyloseq)
library(seqinr)
library(wesanderson)
library(ggplot2)
library(vegan)
library(dplyr)
library(DESeq2)
library(reshape)

Loading required package: permute

Attaching package: ‘permute’

The following object is masked from ‘package:seqinr’:

    getType

Loading required package: lattice
This is vegan 2.4-4

Attaching package: ‘dplyr’

The following object is masked from ‘package:seqinr’:

    count

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘

In [31]:
track = data.frame(readRDS("../Dada2_Results_Pooled_Full/track.rds"))
track$names=row.names(track)

In [32]:
track = track%>%
    arrange(input)
track = track[16:dim(track)[1],]
head(track)

,input,filtered,denoised,tabled,nonchim,names
16,31838,23599,18144,18144,14606,CNL_067_AACGCTGA-CTACTATA_
17,34886,28606,23383,23383,20044,CNL_062_ACGTGCGC-CGTGAGTG_
18,34893,27508,18993,18993,16204,CNL_088_TCTCTATG-ACTATCTG_
19,35177,24124,18701,18701,17543,CNL_029_CGTAGCGA-ACGACGTG_
20,35526,28036,19739,19739,16923,CNL_090_TCTCTATG-CGTTACTA_
21,35926,28406,21665,21665,18806,CNL_106_CGAGCGAC-ACTATCTG_


In [33]:
median(track$input)
min(track$input)
max(track$input)

[1] 58142

[1] 31838

[1] 208448

In [36]:
mean(track$filtered/track$input)
mean(track$denoised/track$input)
mean(track$nonchim/track$input)

[1] 0.7956231

[1] 0.6357043

[1] 0.5306604

In [38]:
median(track$nonchim)
min(track$nonchim)
max(track$nonchim)

[1] 29141

[1] 14606

[1] 180274

In [2]:
# Bring in sequence names from fasta file for now
fasta = read.fasta(file="../Dada2_Results_Pooled_Full/DADA2_seqs_nochim.fasta",seqtype = c("DNA"))
SeqNames = sapply(strsplit(getName(fasta),";"), `[`, 1)

In [176]:
# Import OTU table R object
OTUtab = readRDS("../Dada2_Results_Pooled_Full/OTUtab.nochim.rds")

In [177]:
# Classify it to phyloseq OTUtab
OTUtab = otu_table(OTUtab, taxa_are_rows=FALSE)

# Get the initial sample names
SamNames = sample_names(OTUtab)
# Split out sample names to get just sample ID part, not full sequencing output
SamNames = data.frame(stringr::str_split_fixed(SamNames, "_", 3)[,1:2])
# Name those elements we kept Set and Number
colnames(SamNames)=c("Set","Number")
# Create new column called Names that joins Set and Number with an underscore
SamNames = SamNames %>%
    mutate(Names=paste(Set,"_",Number,sep=""))
# Set those Names as the sample names
SamNames = SamNames$Names

# Sample 151 got labelled as 157 I'm pretty certain - there is no 151, there are two 157 - fixing this
SamNames[160:161] = c("CNL_151","CNL_157")

# We have two muck samples here - renaming
SamNames[195:196] = c("CNL_MUC-1","CNL_MUC-2")

# We have a re-run of sample 89 (actually, neither worked)
SamNames[92:93] = c("CNL_089-1","CNL_089-2")

# Make sure all sample names are there and unique
length(SamNames)
length(unique(SamNames))

# Rename the OTU table samples with these names that will match sample data files
sample_names(OTUtab)=SamNames

[1] 200

[1] 200

In [178]:
# Pulling in the sample data
samdat = sample_data(read.csv("../data/Soils_data/Sample_metadata.txt",sep="\t",row.names=1,header=TRUE))

In [179]:
head(samdat)

,Tube_ID,Seq_ID,Proj_ID,Soil_Rep_Day,Soil_Rep_Day_Trtmt,Qorpak_ID,Amdmt_mg,Soil_g,Soil_Trtmt,Soil_Rep,⋯,PLFA.actinomycetes,PLFA.fungi,PLFA.fungBactRatio,PLFA.actinoBactRatio,CNratio,Day.1,Soil.CO2.mean.mg.per.gram.soil.cum,Amdmt.CO2.mean.mg.per.mg.amdmt.cum,Soil.CO2.mean.mg.per.gram.soil.C.cum,Amdmt.CO2.mean.mg.per.mg.amdmt.C.cum
CNL_001,CNL_1,1,CNL,BL_A_10,BL_A_10_Blank,1,x,x,BL,BL_A,⋯,NA,NA,NA,NA,NA,#N/A,#N/A,#N/A,#N/A,#N/A
CNL_002,CNL_2,2,CNL,BL_A_26,BL_A_26_Blank,2,x,x,BL,BL_A,⋯,NA,NA,NA,NA,NA,#N/A,#N/A,#N/A,#N/A,#N/A
CNL_003,CNL_3,3,CNL,BL_A_10,BL_A_10_Blank,3,x,x,BL,BL_A,⋯,NA,NA,NA,NA,NA,#N/A,#N/A,#N/A,#N/A,#N/A
CNL_004,CNL_4,4,CNL,BL_A_26,BL_A_26_Blank,4,x,x,BL,BL_A,⋯,NA,NA,NA,NA,NA,#N/A,#N/A,#N/A,#N/A,#N/A
CNL_005,CNL_5,5,CNL,AK_A_10,AK_A_10_Soil,5,x,1,AK,AK_A,⋯,3.646,4.327,0.156046,0.1314869,21.38778,10.26666667,0.899071026,NA,9.585234425,NA
CNL_006,CNL_6,6,CNL,AK_A_26,AK_A_26_Soil,6,x,1,AK,AK_A,⋯,3.646,4.327,0.156046,0.1314869,21.38778,26.13333333,2.017104534,NA,21.50488588,NA


In [180]:
# Prepping the taxonomy
TaxTab = read.table("../Dada2_Results_Pooled_Full/taxonomy.tsv",sep="\t", fill=TRUE,header=TRUE)
# Gets the taxonomy table separated into the name+size, taxonomy, and two scores.
V1split = read.table(textConnection(as.character(TaxTab$Feature.ID)), sep=";",fill=TRUE, header=FALSE)
V1split = V1split[,1:2]
# Split out the otu ID and count
V2split = read.table(textConnection(as.character(TaxTab$Taxon)), sep=";",fill=TRUE, header=FALSE)
# Separate the taxonomy into its phylogenetic levels

# Parsing function to split out the relevant part
parser = function(x){
    if (is.na(x)){
        return("NA")
    }
    else {
    val = unlist(strsplit(paste(x),"__"))[2]
    return(val)
        }
}
# Apply to each cell of taxonomy table
V2split = apply(V2split,c(1,2),FUN=parser)

# Give proper row names and column names
row.names(V2split) = V1split$V1
colnames(V2split) = c("Kingdom","Phylum","Class","Order","Family","Genus","Species")
# Convert into phyloseq taxonomy table
TaxTab = tax_table(V2split)
# Make sure names match OTU table (checked via BLAST they were correct)
taxa_names(OTUtab)=taxa_names(TaxTab)

In [181]:
# Bring all elements together for phyloseq object
#ps = phyloseq(OTUtab,samdat,TaxTabPs)
ps = phyloseq(OTUtab,samdat,TaxTab)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 16590 taxa and 197 samples ]
sample_data() Sample Data:       [ 197 samples by 68 sample variables ]
tax_table()   Taxonomy Table:    [ 16590 taxa by 7 taxonomic ranks ]

In [182]:
# Need to get rid of chloroplasts and mitochondria
sum(levels(data.frame(TaxTab)$Phylum) %in% c("Chloroplast", "Mitochondria"))
sum(levels(data.frame(TaxTab)$Class) %in% c("Chloroplast", "Mitochondria"))
sum(levels(data.frame(TaxTab)$Order) %in% c("Chloroplast", "Mitochondria")) # Chloroplast
sum(levels(data.frame(TaxTab)$Family) %in% c("Chloroplast", "Mitochondria")) # Mitochondria
sum(levels(data.frame(TaxTab)$Genus) %in% c("Chloroplast", "Mitochondria"))

[1] 0

[1] 0

[1] 1

[1] 1

[1] 0

In [183]:
ps
ps.pruned = subset_taxa(ps,Order!="Chloroplast")
ps.pruned
ps.pruned = subset_taxa(ps.pruned,Family!="Mitochondria")
ps.pruned
# Removed ~2k taxa. That's good.
ps = ps.pruned

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 16590 taxa and 197 samples ]
sample_data() Sample Data:       [ 197 samples by 68 sample variables ]
tax_table()   Taxonomy Table:    [ 16590 taxa by 7 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 15498 taxa and 197 samples ]
sample_data() Sample Data:       [ 197 samples by 68 sample variables ]
tax_table()   Taxonomy Table:    [ 15498 taxa by 7 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14870 taxa and 197 samples ]
sample_data() Sample Data:       [ 197 samples by 68 sample variables ]
tax_table()   Taxonomy Table:    [ 14870 taxa by 7 taxonomic ranks ]

In [185]:
# Look at total sequence depth for each sample
sort(sample_sums(ps))[1:20]

CNL_089-2  CNL_089-1    CNL_004    CNL_002    CNL_125    CNL_003    CNL_001 
         8         20         36         41         59         83         88 
   CNL_126   CNL_Bm-2   CNL_Bm-3 CNL_PyOM-3 CNL_PyOM-1 CNL_PyOM-2  CNL_MUC-2 
       144        708       1921       2026       2294       4189       4219 
CNL_PyOM-4   CNL_Bm-1   CNL_Bm-4    CNL_067    CNL_088    CNL_090 
      4979       7177      11667      14101      14418      15189

In [186]:
cutoff = 12000
# All blanks are below this value
# Biomass (Bm) samples were previously mostly chloroplasts!
# Omit samples that sequenced poorly
ps = prune_samples(sample_sums(ps)>=cutoff, ps)

# Remove the Muck samples from Johannes
ps = prune_samples(sample_data(ps)$Soil_Trtmt!="Muck", ps)


#ps = subset_taxa(ps,Phylum != "")

In [187]:
# Formatting of sample variables, ordering, etc.
sample_data(ps)$Soil_Trtmt = as.factor(sample_data(ps)$Soil_Trtmt)
sample_data(ps)$Amdmt = as.factor(sample_data(ps)$Amdmt)
sample_data(ps)$Day = as.factor(sample_data(ps)$Day)
sample_data(ps)$Day.1 = as.numeric(sample_data(ps)$Day.1)
sample_data(ps)$Amdmt.CO2.mean.mg.per.mg.amdmt.cum = as.numeric(paste(sample_data(ps)$Amdmt.CO2.mean.mg.per.mg.amdmt.cum))
sample_data(ps)$Soil.CO2.mean.mg.per.gram.soil.C.cum = as.numeric(paste(sample_data(ps)$Soil.CO2.mean.mg.per.gram.soil.C.cum))
sample_data(ps)$Amdmt.CO2.mean.mg.per.mg.amdmt.C.cum = as.numeric(paste(sample_data(ps)$Amdmt.CO2.mean.mg.per.mg.amdmt.C.cum))
sample_data(ps)$Soil.CO2.mean.mg.per.gram.soil.cum = as.numeric(paste(sample_data(ps)$Soil.CO2.mean.mg.per.gram.soil.cum))
levels(sample_data(ps)$Soil_Trtmt) = c("Alaska","Florida","Hawaii","New York","Utah")
sample_data(ps)$Soil_Trtmt = factor(sample_data(ps)$Soil_Trtmt, levels = c("Hawaii","Alaska","Utah","New York","Florida"))
sample_data(ps)$Day = factor(sample_data(ps)$Day, levels=c("1","10","26"))
sample_data(ps)$Amdmt = factor(sample_data(ps)$Amdmt, levels = c("Soil","OM","PyOM"))

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [188]:
# Add actual soil names
sample_data(ps)$Soil_Name=sample_data(ps)$Soil_Trtmt
levels(sample_data(ps)$Soil_Name)[levels(sample_data(ps)$Soil_Name)=="Hawaii"] = "Hydrudand"
levels(sample_data(ps)$Soil_Name)[levels(sample_data(ps)$Soil_Name)=="New York"] = "Fragiudept"
levels(sample_data(ps)$Soil_Name)[levels(sample_data(ps)$Soil_Name)=="Alaska"] = "Cryaquept"
levels(sample_data(ps)$Soil_Name)[levels(sample_data(ps)$Soil_Name)=="Utah"] = "Haplocalcid"
levels(sample_data(ps)$Soil_Name)[levels(sample_data(ps)$Soil_Name)=="Florida"] = "Quartzipsamment"
sample_data(ps)$Soil_Name

[1] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
  [5] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
  [9] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
 [13] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
 [17] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
 [21] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [25] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [29] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [33] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [37] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [41] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
 [45] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [49] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [53] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [57] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [61] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [65] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
 [69] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
 [73] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
 [77] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
 [81] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
 [85] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
 [89] Haplocalcid     Haplocalcid     Haplocalcid     Fragiudept     
 [93] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
 [97] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
[101] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
[105] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
[109] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
[113] Fragiudept      Fragiudept      Fragiudept      Cryaquept      
[117] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
[121] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
[125] Cryaquept       Cryaquept       Cryaquept       Cryaquept      
[129] Cryaquept       Cryaquept       Cryaquept       Hydrudand      
[133] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
[137] Hydrudand       Hydrudand       Hydrudand       Hydrudand      
[141] Hydrudand       Hydrudand       Hydrudand       Fragiudept     
[145] Fragiudept      Quartzipsamment Quartzipsamment Quartzipsamment
[149] Fragiudept      Quartzipsamment Quartzipsamment Quartzipsamment
[153] Quartzipsamment Quartzipsamment Quartzipsamment Quartzipsamment
[157] Quartzipsamment Quartzipsamment Haplocalcid     Haplocalcid    
[161] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
[165] Haplocalcid     Haplocalcid     Haplocalcid     Haplocalcid    
[169] Haplocalcid     Haplocalcid     Fragiudept      Fragiudept     
[173] Fragiudept      Fragiudept      Fragiudept      Fragiudept     
[177] Fragiudept      Fragiudept      Fragiudept     
Levels: Hydrudand Cryaquept Haplocalcid Fragiudept Quartzipsamment

In [189]:
sample_data(ps)$Soil_Rep_Day

[1] AK_A_10 AK_A_26 AK_B_10 AK_B_26 AK_B_10 AK_B_26 AK_B_10 AK_B_26 AK_C_10
 [10] AK_C_26 AK_C_10 AK_C_26 AK_C_10 AK_C_26 AK_D_10 AK_D_26 AK_D_10 AK_D_26
 [19] AK_D_10 AK_D_26 FL_A_10 FL_A_26 FL_A_10 FL_A_26 FL_A_10 FL_A_26 FL_B_10
 [28] FL_B_26 FL_B_10 FL_B_26 FL_B_10 FL_B_26 FL_C_10 FL_C_26 FL_C_10 FL_C_26
 [37] FL_C_10 FL_C_26 FL_D_10 FL_D_26 FL_D_10 FL_D_26 FL_D_10 FL_D_26 HI_A_10
 [46] HI_A_26 HI_A_10 HI_A_26 HI_A_10 HI_A_26 HI_B_10 HI_B_26 HI_B_10 HI_B_26
 [55] HI_B_10 HI_B_26 HI_C_10 HI_C_26 HI_C_10 HI_C_26 HI_C_10 HI_C_26 HI_D_10
 [64] HI_D_26 HI_D_10 HI_D_26 HI_D_10 HI_D_26 UT_A_10 UT_A_26 UT_A_10 UT_A_26
 [73] UT_A_10 UT_A_26 UT_B_10 UT_B_26 UT_B_10 UT_B_26 UT_B_10 UT_B_26 UT_C_10
 [82] UT_C_26 UT_C_10 UT_C_26 UT_C_26 UT_D_10 UT_D_26 UT_D_10 UT_D_26 UT_D_10
 [91] UT_D_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26
[100] MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10
[109] MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 MP_A_10 MP_A_26 AK_A_10 AK_A_26
[118] AK_A_10 AK_A_26 AK_A_1  AK_A_1  AK_A_1  AK_B_1  AK_B_1  AK_B_1  AK_C_1 
[127] AK_C_1  AK_C_1  AK_D_1  AK_D_1  AK_D_1  HI_A_1  HI_A_1  HI_A_1  HI_B_1 
[136] HI_B_1  HI_B_1  HI_C_1  HI_C_1  HI_C_1  HI_D_1  HI_D_1  HI_D_1  MP_A_1 
[145] MP_A_1  FL_A_1  FL_A_1  FL_A_1  MP_A_1  FL_B_1  FL_B_1  FL_B_1  FL_C_1 
[154] FL_C_1  FL_C_1  FL_D_1  FL_D_1  FL_D_1  UT_A_1  UT_A_1  UT_A_1  UT_B_1 
[163] UT_B_1  UT_B_1  UT_C_1  UT_C_1  UT_C_1  UT_D_1  UT_D_1  UT_D_1  MP_A_1 
[172] MP_A_1  MP_A_1  MP_A_1  MP_A_1  MP_A_1  MP_A_1  MP_A_1  MP_A_1 
51 Levels: AK_A_1 AK_A_10 AK_A_26 AK_B_1 AK_B_10 AK_B_26 AK_C_1 ... UT_D_26

In [190]:
# Remove FL sample that's way off from rest of FL and its corresponding PyOM and OM
ps
ps = prune_samples(!(sample_data(ps)$Soil_Rep_Day=="FL_D_26"),ps)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14870 taxa and 179 samples ]
sample_data() Sample Data:       [ 179 samples by 69 sample variables ]
tax_table()   Taxonomy Table:    [ 14870 taxa by 7 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14870 taxa and 176 samples ]
sample_data() Sample Data:       [ 176 samples by 69 sample variables ]
tax_table()   Taxonomy Table:    [ 14870 taxa by 7 taxonomic ranks ]

In [191]:
# Sample 89 didn't sequence properly either time and was filtered out.
# We will remove that set of samples (the corresponding PyOM and OM treatments)
ps
ps = prune_samples(!(sample_data(ps)$Soil_Rep_Day=="UT_C_10"),ps)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14870 taxa and 176 samples ]
sample_data() Sample Data:       [ 176 samples by 69 sample variables ]
tax_table()   Taxonomy Table:    [ 14870 taxa by 7 taxonomic ranks ]

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 14870 taxa and 174 samples ]
sample_data() Sample Data:       [ 174 samples by 69 sample variables ]
tax_table()   Taxonomy Table:    [ 14870 taxa by 7 taxonomic ranks ]

In [192]:
ps.norm = transform_sample_counts(ps, function(x) x / sum(x) )
# Regular relative abundance
ps.hell = transform_sample_counts(ps, function(x) (x / sum(x))^0.5 )
# Hellinger transformation

In [193]:
df = sample_data(ps.norm)
df = df %>%
    group_by(Soil_Name,Day, Amdmt)%>%
    summarize(n())
df

Soil_Name,Day,Amdmt,n()
Hydrudand,1,Soil,4
Hydrudand,1,OM,4
Hydrudand,1,PyOM,4
Hydrudand,10,Soil,4
Hydrudand,10,OM,4
Hydrudand,10,PyOM,4
Hydrudand,26,Soil,4
Hydrudand,26,OM,4
Hydrudand,26,PyOM,4
Cryaquept,1,Soil,4


In [194]:
# Save these objects for future notebooks

#saveRDS(ps,"../data/Cornell16S/ps.16S")
#saveRDS(ps.norm,"../data/Cornell16S/ps.16S.norm")
#saveRDS(ps.hell,"../data/Cornell16S/ps.16S.hell")